In [2]:
from sage.all import *
import numpy as np

In [114]:
x0 = 0 # initial condition
y0 = 1200 # initial condition

last_x = 480 # last x value or x value at which we need approximation

x = var('x')
y = function('y')(x)
de = diff(y, x, 1) == -2.2067 * 1e-12 * (y ** 4 - 81 * 10 ** 8)
f = desolve(de, y, [x0, y0])

x, y = var('x, y')
dydx = -2.2067 * 1e-12 * (y ** 4 - 81 * 10 ** 8)
firstOrderDiffOf_dydx = dydx.diff(x) + dydx.diff(y)*dydx
secondOrderDiffOf_dydx = (dydx.diff(x) + dydx.diff(y)*dydx).diff(x) + (dydx.diff(x) + dydx.diff(y)*dydx).diff(y)*dydx

def DyDx(x0, y0):
    return RR(dydx.subs({x: x0, y: y0}))

# print("Real value at x = {} is {}".format(last_x, real_value))

def printTable(listxy):
    print("x\t\ty")
    for i in listxy:
        print("{}\t\t{}".format(i[0], i[1]))

def Output(approximateValue, absoluteError, approximateError=""):
    print("Real value = {}".format(real_value))
    print("Approximate value = {}".format(approximateValue))
    print("Absolute error = {}".format(absoluteError))
    if approximateError != "":
        print("Approximate error = {}".format(approximateError))


In [115]:
# euler method
h = 0.5 # step size

def euler(x0, y0, h, last_x):
    listxy = []
    xi = x0
    yi = y0
    listxy.append((xi, yi))
    while xi < last_x:
        yi = yi + h * DyDx(xi, yi)
        xi = xi + h
        listxy.append((xi, yi))
    return listxy

def approximate_error_euler_method(last_x, approximate_y, h):
    return RR(firstOrderDiffOf_dydx.subs({ x: last_x, y: approximate_y })) * h ** 3 / 6 + RR(secondOrderDiffOf_dydx.subs({ x: last_x, y: approximate_y })) * h ** 4 / 24

listxy = euler(x0, y0, h, last_x)
approximate_value = listxy[-1][1]
print(approximate_value)
# absolute_error = abs(real_value - approximate_value)
# approximateError = approximate_error_euler_method(last_x, approximate_value, h)
# Output(approximate_value, absolute_error, approximateError)


647.340376070441


In [125]:
# runge kuta 2nd method

# heun method a2 = 1/2
def heun(x0, y0, h, last_x):
    listxy = []
    xi = x0
    yi = y0
    listxy.append((xi, yi))
    while xi < last_x:
        k1 = DyDx(xi, yi)
        k2 = DyDx(xi + h, yi + h * k1)
        yi = yi + h * (0.5 * k1 + 0.5 * k2)
        xi = xi + h
        listxy.append((xi, yi))
    return listxy

# midpoint method a2 = 1
def midpoint(x0, y0, h, last_x):
    listxy = []
    xi = x0
    yi = y0
    listxy.append((xi, yi))
    while xi < last_x:
        k1 = DyDx(xi, yi)
        k2 = DyDx(xi + 0.5 * h, yi + h * k1 * 0.5)
        yi = yi + h * k2
        xi = xi + h
        listxy.append((xi, yi))
    return listxy

# ralston method a2 = 3/4
def ralston(x0, y0, h, last_x):
    listxy = []
    xi = x0
    yi = y0
    listxy.append((xi, yi))
    while xi < last_x:
        k1 = DyDx(xi, yi)
        k2 = DyDx(xi + 0.75 * h, yi + h * k1 * 0.75)
        yi = yi + h * (2 * k2 + k1) / 3
        xi = xi + h
        listxy.append((xi, yi))
    return listxy

def popular_runge_kuta_4th_method(x0, y0, h, last_x):
    listxy = []
    xi = x0
    yi = y0
    listxy.append((xi, yi))
    while xi < last_x:
        k1 = DyDx(xi, yi)
        k2 = DyDx(xi + 0.5 * h, yi + h * k1 * 0.5)
        k3 = DyDx(xi + 0.5 * h, yi + h * k2 * 0.5)
        k4 = DyDx(xi + h, yi + h * k3)
        yi = yi + h * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        xi = xi + h
        listxy.append((xi, yi))
    return listxy

h = 30 # step size

listxy = popular_runge_kuta_4th_method(x0, y0, h, last_x)
approximate_value = listxy[-1][1]
print(approximate_value)
# absolute_error = abs(real_value - approximate_value)
# Output(approximate_value, absolute_error)
printTable(listxy)

647.572053691959
x		y
0		1200
30		1088.04242484851
60		1008.95226927830
90		948.888384368908
120		901.072966031023
150		861.723562404048
180		828.532458631230
210		799.997060910085
240		775.089168847912
270		753.077041396428
300		733.423086347404
330		715.721812626502
360		699.660518022618
390		684.993453511100
420		671.524308336466
450		659.094014251559
480		647.572053691959


In [178]:
# shooting method
x, y, z = var('x, y, z')

dydx = z
dzdx = (- z * x + y) / (x ** 2)

def DYDX(x0, y0, z0):
    return RR(dydx.subs({x: x0, y: y0, z: z0}))

def DZDX(x0, y0, z0):
    return RR(dzdx.subs({x: x0, y: y0, z: z0}))

def shooting_method_euler(x0, y0, z0, h, x1):
    listxyz = []
    listxyz.append((x0, y0, z0))
    xi = x0
    yi = y0
    zi = z0
    while xi < x1:
        temp_yi = yi + h * DYDX(xi, yi, zi)
        temp_zi = zi + h * DZDX(xi, yi, zi)
        temp_xi = xi + h
        
        yi = temp_yi
        zi = temp_zi
        xi = temp_xi
        listxyz.append((xi, yi, zi))
    return listxyz

def printListXYZ(listxyz):
    print("x\t\ty\t\tz")
    for i in listxyz:
        print("{}\t\t{}\t\t{}".format(i[0], i[1], i[2]))
        
h = 0.75

x0 = 5
y0 = 0.0038731

x1 = 8
y1 = 0.0030770

# assume z0
z01 = (y1 - y0) / (x1 - x0)

hasil1 = shooting_method_euler(x0, y0, z01, h, x1)

z02 = 2 * z01

hasil2 = shooting_method_euler(x0, y0, z02, h, x1)

printListXYZ(hasil1)
printListXYZ(hasil2)

approximate1 = hasil1[-1][1]
approximate2 = hasil2[-1][1]

z03 = z01 + (z02 - z01) * (y1 - approximate1) / (approximate2 - approximate1)
print(z03)

printListXYZ(shooting_method_euler(x0, y0, z03, h, x1))

x		y		z
5		0.00387310000000000		-0.000265366666666667
5.75000000000000		0.00367407500000000		-0.000109368666666667
6.50000000000000		0.00359204850000000		-0.0000117593320730939
7.25000000000000		0.00358322900094518		0.0000533616884915530
8.00000000000000		0.00362325026731384		0.0000989696327395638
x		y		z
5		0.00387310000000000		-0.000530733333333333
5.75000000000000		0.00347505000000000		-0.000334930333333333
6.50000000000000		0.00322385225000000		-0.000212414656584751
7.25000000000000		0.00306454125756144		-0.000130677126682960
8.00000000000000		0.00296653341254922		-0.0000734316984877704
-0.000486095907207941
x		y		z
5		0.00387310000000000		-0.000486095907207941
5.75000000000000		0.00350852806959404		-0.000296988521126750
6.50000000000000		0.00328578667874898		-0.000178662349495607
7.25000000000000		0.00315178991662728		-0.0000997198888363691
8.00000000000000		0.00307700000000000		-0.0000444320061630142


In [192]:
# finite difference method
h = 0.6

x0 = 5
y0 = 0.0038731

x1 = 8
y1 = 0.0030770

x = var('x')
right_side = 0

def finiteDifferenceMethod(x0, y0, x1, y1, h):
    n = int((x1 - x0) / h + 1)
    mat = matrix(RR, n, n, 0)
    xi_min1_function = 1 / (h)**2
    xi_function1 = - 2 / h**2 - 1 / (x * h) - 1 / (x ** 2)
    xi_plus1_function = 1 / h**2 + 1 / (x * h)
    mat[0, 0] = 1
    xi = x0
    for i in range(1, n - 1):
        xi = xi + h
        mat[i, i - 1] = xi_min1_function.subs({x: xi})
        mat[i, i] = xi_function1.subs({x: xi})
        mat[i, i + 1] = xi_plus1_function.subs({x: xi})
    mat[n - 1, n - 1] = 1
    
    vektor = vector(RR, n)
    vektor[0] = y0
    vektor[n - 1] = y1
    for i in range(1, n - 1):
        vektor[i] = right_side.subs({x: x0 + i * h})
    print(mat)
    print(vektor)
    return mat.solve_right(vektor)

finiteDifferenceMethod(x0, y0, x1, y1, h)

[ 1.00000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[ 2.77777777777778 -5.88506235827664  3.07539682539683 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000  2.77777777777778 -5.85038732801480  3.04659498207885 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000  2.77777777777778 -5.82227989234910  3.02287581699346 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000  2.77777777777778 -5.79904228552877  3.00300300300300]
[0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000  1.00000000000000]
(0.00387310000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.00307700000000000)


(0.00387310000000000, 0.00361649726466179, 0.00342222529562585, 0.00327431700854530, 0.00316182644671047, 0.00307700000000000)